# Proyecto 3 - G3 - ML

Introducción.....

## 0. Libraries

In [3]:
import os
import glob

import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rcParams
from sklearn.decomposition import PCA

np.random.seed(2024)

## 1. Video Features Extracion

Para la extracción de features se usó la librería **Video Features**. No se escogió **Gluon** porque no ha tenido mantenimiento desde hace más de 1 año. 

Modelos disponibles: 

- S3D (Kinetics 400)
- R(2+1)d RGB (IG-65M, Kinetics 400):
    - r2plus1d_18_16_kinetics
    - r2plus1d_34_32_ig65m_ft_kinetics *
    - r2plus1d_34_8_ig65m_ft_kinetics
- I3D-Net RGB + Flow (Kinetics 400)

Al final obtuvimos un conjunto de .npy, los cuales leeremos y 

El comando usado fue el siguiente:

```bash
# para usar con cpu agregar argumento
python .... device="cpu"

# i3d
python main.py feature_type=i3d file_with_video_paths="../data/train_subset/video_paths.txt" on_extraction=save_numpy output_path="../data/train_output"

# s3d
python main.py feature_type=s3d file_with_video_paths="../data/train_subset/video_paths.txt" on_extraction=save_numpy output_path="../data/train_output"

# r2plus1d_18_16_kinetics
python main.py feature_type=r21d file_with_video_paths="../data/train_subset/video_paths.txt" model_name="r2plus1d_18_16_kinetics" on_extraction=save_numpy output_path="../data/train_output"

# r2plus1d_34_32_ig65m_ft_kinetics
python main.py feature_type=r21d file_with_video_paths="../data/train_subset/video_paths.txt" model_name="r2plus1d_34_32_ig65m_ft_kinetics" on_extraction=save_numpy output_path="../data/train_output"

# r2plus1d_34_8_ig65m_ft_kinetics
python main.py feature_type=r21d file_with_video_paths="../data/train_subset/video_paths.txt" model_name="r2plus1d_34_8_ig65m_ft_kinetics" on_extraction=save_numpy output_path="../data/train_output"
```

In [5]:
def split_files_to_groups(directory, num_groups=5):
    
    # Lista de los .mp4
    mp4_files = [f for f in os.listdir(directory) if f.endswith('.mp4')]
    # Concatenar path
    mp4_paths = [os.path.join(directory, f) for f in mp4_files]
    # Shuffle
    np.random.shuffle(mp4_paths)

    # Dividir en grupos
    groups = [[] for _ in range(num_groups)]
    for i, path in enumerate(mp4_paths):
        groups[i % num_groups].append(path)

    # Escribir
    for i, group in enumerate(groups):
        with open(f'group_{i+1}.txt', 'w') as file:
            for path in group:
                file.write(f"{path}\n")
                
directory = 'C:\\Users\\Adrian Cespedes\\Desktop\\utec2024_1\\ml_temp\\Proy3-ML\\data\\temp'
split_files_to_groups(directory)

# 2. Load & Save Feature Vectors